# Variables and Memory

Notes taken from Udacity Cpp Nanodregree

<img width="400" src="figs/virtualmemory.png">

** Figure 1: Virtual Memory **



* The **stack** is a contiguous memory block with a fixed maximum size. If a program exceeds this size, it will crash. The stack is used for storing automatically allocated variables such as local variables or function parameters. If there are multiple threads in a program, then each thread has its own stack memory. New memory on the stack is allocated when the path of execution enters a scope and freed again once the scope is left. It is important to know that the stack is managed "automatically" by the compiler, which means we do not have to concern ourselves with allocation and deallocation.

* The **heap** (also called "free store" in C++) is where data with dynamic storage lives. It is shared among multiple threads in a program, which means that memory management for the heap needs to take concurrency into account. This makes memory allocations in the heap more complicated than stack allocations. In general, managing memory on the heap is more (computationally) expensive for the operating system, which makes it slower than stack memory. Contrary to the stack, the heap is not managed automatically by the system, but by the programmer. If memory is allocated on the heap, it is the programmer’s responsibility to free it again when it is no longer needed. If the programmer manages the heap poorly or not at all, there will be trouble.

* The **BSS** (Block Started by Symbol) segment is used in many compilers and linkers for a segment that contains global and static variables that are initialized with zero values. This memory area is suitable, for example, for arrays that are not initialized with predefined values.

* The **Data segment** serves the same purpose as the BSS segment with the major difference being that variables in the Data segment have been initialized with a value other than zero. Memory for variables in the Data segment (and in BSS) is allocated once when a program is run and persists throughout its lifetime.


### Memory allocation

* **Static memory allocation** is performed for static and global variables, which are stored in the BSS and Data segment. Memory for these types of variables is allocated once when your program is run and persists throughout the life of your program.

* **Automatic memory allocation** is performed for function parameters as well as local variables, which are stored on the stack. Memory for these types of variables is allocated when the path of execution enters a scope and freed again once the scope is left.

* **Dynamic memory allocation** is a possibility for programs to request memory from the operating system at runtime when needed. This is the major difference to automatic and static allocation, where the size of the variable must be known at compile time. Dynamic memory allocation is not performed on the limited stack but on the heap and is thus (almost) only limited by the size of the address space.


# Properties of Stack Memory

* The stack is a **contiguous block of memory**. It will not become fragmented (as opposed to the heap) and it has a fixed maximum size.

* When the **maximum size of the stack** memory is exceeded, a program will crash.

* Allocating and deallocating **memory is fast** on the stack. It only involves moving the stack pointer to a new position.

* When using **multiple threads** (as in concurrent programming), it is important to know that **each thread has its own stack memory** - which can be considered thread-safe. 


### Example 01 Stack Growth and Contraction

In this example we can see that memory is contigouos for i = 1, 2, 4. But for function "MyFunc()"  it uses a part of the memory that is before the main() declaration

```
1: 0x7fff20c3f53c 
2: 0x7fff20c3f540 
3: 0x7fff20c3f514 -> MyFunc()
4: 0x7fff20c3f544 
``` 

Between 2 and 3, the address pointer is moved by 0x28.
MyFunc, the compiler needs to store additional data such as the return address. 


<img width="400" src="figs/growthandcontracts.png">

** Figure 1: Stack Growth and Contracts **

When a thread is created, stack memory is allocated by the operating system as a contiguous block. With each new function call or local variable allocation, the stack pointer is moved until eventually it will reach the bottom of said memory block. Once it exceeds this limit (which is called "stack overflow"), the program will crash. We will try to find out the limit of your computer’s stack memory in the following exercise.

### Pointers vs. References


As we have seen in the examples above, the use of pointers and references to directly manipulate function arguments in a memory-effective way is very similar. Let us compare the two methods in the code on the right.

As can be seen, pointer and reference are both implemented by using a memory address. In the case of AddFour the caller does not even realize that val might be modified while with AddSix, the reference to val has to be explicitly written by using &.

If passing by value needs to be avoided, both pointers and references are a way to achieve this. The following selection of properties contrasts the two methods so it will be easier to decide which to use from the perspective of the use-case at hand:

* Pointers can be declared without initialization. This means we can pass an uninitialized pointer to a function who then internally performs the initialization for us.

* Pointers can be reassigned to another memory block on the heap.

* References are usually easier to use (depending on the expertise level of the programmer). Sometimes however, if a third-party function is used without properly looking at the parameter definition, it might go unnoticed that a value has been modified.



Now, we will compare the three strategies we have seen so far with regard to stack memory usage. Consider the code on the right.

After creating a local variable i in main to give us the address of the stack bottom, we are passing i by-value to our first function. Inside CallByValue, the memory address of a local variable j is printed to the console, which serves as a marker for the stack pointer. With the second function call in main, we are passing a reference to i to CallByPointer. Lastly, the function CallByReference is called in main, which again takes the integer i as an argument. However, from looking at main alone, we can not tell wether i will be passed by value or by reference.

On my machine, when compiled with g++ (Apple clang version 11.0.0), the program produces the following output:

    stack bottom: 0x7ffeefbff698
    call-by-value: 0x7ffeefbff678
    call-by-pointer: 0x7ffeefbff674
    call-by-reference: 0x7ffeefbff674

Depending on your system, the compiler you use and the compiler optimization techniques, you man not always see this result. In some cases

Let us take a look at the respective differences to the stack bottom in turn:

* CallByValue requires 32 bytes of memory. As discussed before, this is reserved for e.g. the function return address and for the local variables within the function (including the copy of i).

* CallByPointer on the other hand requires - perhaps surprisingly - 36 bytes of memory. Let us complete the examination before going into more details on this result.

Let us take a look at the size of the various parameter types using the sizeof command:

    printf("size of int: %lu\n", sizeof(int));
    printf("size of *int: %lu\n", sizeof(int *));

The output here is

    size of int: 4
    size of *int: 8

Obviously, the size of the pointer variable is larger than the actual data type. As my machine has a 64 bit architecture, an address requires 8 byte.

As an experiment, you could use the -m32 compiler flag to build a 32 bit version of the program. This yields the following output:

    size of int: 4
    size of *int: 4

In order to benefit from call-by-reference, the size of the data type 


# Heap